<a href="https://colab.research.google.com/github/martialaristide/AGRICAM-IA-/blob/main/Syst%C3%A8me_de_D%C3%A9tection_M%C3%A9dicale.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import cv2
from PIL import Image
import json
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.model_selection import train_test_split

In [35]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms, models
from torchvision.utils import make_grid
import torchvision.transforms.functional as TF

In [36]:
import pydicom
import nibabel as nib


torch.manual_seed(42)
np.random.seed(42)

print(" Toutes les bibliothèques importées avec succès!")

 Toutes les bibliothèques importées avec succès!


In [37]:
!pip install pydicom


In [38]:
import pydicom
import nibabel as nib

In [39]:
torch.manual_seed(42)
np.random.seed(42)

In [40]:
print(" Toutes les bibliothèques importées avec succès!")

 Toutes les bibliothèques importées avec succès!


In [41]:
class MedicalDataLoader:
    """
    Classe pour charger et préparer les données médicales
    """

    def __init__(self, data_path):
        self.data_path = data_path
        self.supported_formats = ['.dcm', '.nii', '.nii.gz', '.png', '.jpg']

    def load_dicom(self, file_path):
        """Charge un fichier DICOM (radiographies)"""
        try:
            dicom = pydicom.dcmread(file_path)
            image = dicom.pixel_array
            image = image.astype(np.float32) / np.max(image)
            return image, dicom
        except Exception as e:
            print(f"Erreur lecture DICOM {file_path}: {e}")
            return None, None

    def load_nifti(self, file_path):
        """Charge un fichier NIFTI (IRM)"""
        try:
            nifti = nib.load(file_path)
            image = nifti.get_fdata()
            return image, nifti
        except Exception as e:
            print(f"Erreur lecture NIFTI {file_path}: {e}")
            return None, None

    def explore_dataset(self):
        """Explore la structure du dataset"""
        print(" Exploration du dataset médical...")

        file_types = {}
        for root, dirs, files in os.walk(self.data_path):
            for file in files:
                ext = os.path.splitext(file)[1].lower()
                if ext in self.supported_formats:
                    file_types[ext] = file_types.get(ext, 0) + 1

        print(" Distribution des fichiers:", file_types)


        self.preview_medical_images()

    def preview_medical_images(self):
        """Placeholder for previewing medical images - functionality to be implemented"""
        print(" Previewing medical images (functionality to be implemented)...")

data_loader = MedicalDataLoader('./medical_data/')
data_loader.explore_dataset()

 Exploration du dataset médical...
 Distribution des fichiers: {}
 Previewing medical images (functionality to be implemented)...


In [42]:
class MedicalClassifier(nn.Module):
    """
    CNN pour classification de pathologies sur images radiologiques
    Utilise transfer learning avec ResNet50
    """

    def __init__(self, num_classes=2, pretrained=True):
        super(MedicalClassifier, self).__init__()

        self.backbone = models.resnet50(pretrained=pretrained)

        in_features = self.backbone.fc.in_features
        self.backbone.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(in_features, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        return self.backbone(x)

print(" Modèle de classification créé!")

 Modèle de classification créé!


In [43]:
class UNetSegmentation(nn.Module):
    """
    Architecture U-Net pour segmentation de tumeurs en IRM
    Spécialement conçu pour l'imagerie médicale
    """

    def __init__(self, in_channels=1, out_channels=1, features=[64, 128, 256, 512]):
        super(UNetSegmentation, self).__init__()

        self.encoder = nn.ModuleList()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        for feature in features:
            self.encoder.append(self._double_conv(in_channels, feature))
            in_channels = feature

        self.bottleneck = self._double_conv(features[-1], features[-1] * 2)

        self.decoder = nn.ModuleList()
        self.upconvs = nn.ModuleList()

        for feature in reversed(features):
            self.upconvs.append(
                nn.ConvTranspose2d(feature * 2, feature, kernel_size=2, stride=2)
            )
            self.decoder.append(self._double_conv(feature * 2, feature))

        self.final_conv = nn.Conv2d(features[0], out_channels, kernel_size=1)

    def _double_conv(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, 3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        skip_connections = []

        for down in self.encoder:
            x = down(x)
            skip_connections.append(x)
            x = self.pool(x)

        x = self.bottleneck(x)
        skip_connections = skip_connections[::-1]

        for idx in range(len(self.decoder)):
            x = self.upconvs[idx](x)
            skip_connection = skip_connections[idx]


            if x.shape != skip_connection.shape:
                x = TF.resize(x, size=skip_connection.shape[2:])

            concat_skip = torch.cat((skip_connection, x), dim=1)
            x = self.decoder[idx](concat_skip)

        return torch.sigmoid(self.final_conv(x))

print(" Modèle de segmentation U-Net créé!")

 Modèle de segmentation U-Net créé!


In [44]:
class MedicalDataset(Dataset):
    """
    Dataset personnalisé pour données médicales
    Gère DICOM, NIFTI et formats standards
    """

    def __init__(self, image_paths, masks_paths=None, transform=None,
                 mode='classification', image_size=256):
        self.image_paths = image_paths
        self.masks_paths = masks_paths
        self.transform = transform
        self.mode = mode
        self.image_size = image_size
        self.data_loader = MedicalDataLoader('')

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = self.load_medical_image(img_path)

        if self.mode == 'classification':
            label = self.get_label_from_path(img_path)
            if self.transform:
                image = self.transform(image)
            return image, label

        elif self.mode == 'segmentation':
            mask_path = self.masks_paths[idx]
            mask = self.load_medical_image(mask_path)

            if self.transform:
                augmented = self.transform(image=image, mask=mask)
                image, mask = augmented['image'], augmented['mask']

            return image, mask

    def load_medical_image(self, path):
        """Charge une image médicale selon son format"""
        ext = os.path.splitext(path)[1].lower()

        if ext == '.dcm':
            image, _ = self.data_loader.load_dicom(path)
        elif ext in ['.nii', '.nii.gz']:
            image, _ = self.data_loader.load_nifti(path)
        else:
            image = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
            image = image.astype(np.float32) / 255.0

        if len(image.shape) == 3:
            image = image[:, :, image.shape[2]//2]

        image = cv2.resize(image, (self.image_size, self.image_size))
        return image

    def get_label_from_path(self, path):
        """Extrait le label du chemin (à adapter selon la structure)"""
        if 'normal' in path.lower():
            return 0
        elif 'cancer' in path.lower() or 'tumor' in path.lower():
            return 1
        else:
            return 0

print(" Dataset médical créé!")

 Dataset médical créé!


In [45]:
class MedicalTrainer:
    """
    Classe pour l'entraînement des modèles médicaux
    """

    def __init__(self, model, device, model_type='classification'):
        self.model = model
        self.device = device
        self.model_type = model_type
        self.history = {'train_loss': [], 'val_loss': [], 'train_acc': [], 'val_acc': []}

    def train_classification(self, train_loader, val_loader, criterion,
                           optimizer, scheduler, epochs=100):
        """
        Entraînement pour la classification
        """
        print(" Début de l'entraînement classification...")

        best_val_acc = 0.0

        for epoch in range(epochs):
            self.model.train()
            train_loss = 0.0
            train_correct = 0
            train_total = 0

            for images, labels in tqdm(train_loader, desc=f'Epoch {epoch+1}/{epochs}'):
                images, labels = images.to(self.device), labels.to(self.device)

                optimizer.zero_grad()
                outputs = self.model(images)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

                train_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                train_total += labels.size(0)
                train_correct += (predicted == labels).sum().item()

            val_loss, val_acc = self.validate_classification(val_loader, criterion)

            self.history['train_loss'].append(train_loss/len(train_loader))
            self.history['val_loss'].append(val_loss)
            self.history['train_acc'].append(100.*train_correct/train_total)
            self.history['val_acc'].append(val_acc)

            if scheduler:
                scheduler.step()

            if val_acc > best_val_acc:
                best_val_acc = val_acc
                torch.save(self.model.state_dict(), 'best_classification_model.pth')

            print(f'Epoch {epoch+1}/{epochs}:')
            print(f'  Train Loss: {train_loss/len(train_loader):.4f}, Acc: {100.*train_correct/train_total:.2f}%')
            print(f'  Val Loss: {val_loss:.4f}, Acc: {val_acc:.2f}%')

    def validate_classification(self, val_loader, criterion):
        """Validation pour la classification"""
        self.model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(self.device), labels.to(self.device)
                outputs = self.model(images)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

        return val_loss/len(val_loader), 100.*val_correct/val_total

print(" Trainer médical créé!")

 Trainer médical créé!


In [46]:
class MedicalEvaluator:
    """
    Évaluation spécifique pour modèles médicaux
    """

    def __init__(self, model, device):
        self.model = model
        self.device = device

    def evaluate_classification(self, test_loader):
        """Évaluation complète pour la classification"""
        self.model.eval()
        all_predictions = []
        all_labels = []
        all_probabilities = []

        with torch.no_grad():
            for images, labels in test_loader:
                images = images.to(self.device)
                outputs = self.model(images)
                probabilities = F.softmax(outputs, dim=1)
                _, predictions = torch.max(outputs, 1)

                all_predictions.extend(predictions.cpu().numpy())
                all_labels.extend(labels.numpy())
                all_probabilities.extend(probabilities.cpu().numpy())

        self.calculate_medical_metrics(all_labels, all_predictions, all_probabilities)

    def calculate_medical_metrics(self, true_labels, predictions, probabilities):
        """Calcule les métriques spécifiques au domaine médical"""

        print(" Classification Report:")
        print(classification_report(true_labels, predictions, target_names=['Normal', 'Pathologique']))

        cm = confusion_matrix(true_labels, predictions)
        tn, fp, fn, tp = cm.ravel()

        sensitivity = tp / (tp + fn)
        specificity = tn / (tn + fp)
        precision = tp / (tp + fp)
        f1_score = 2 * (precision * sensitivity) / (precision + sensitivity)

        auc_roc = roc_auc_score(true_labels, [p[1] for p in probabilities])

        print("\n🏥 Métriques Médicales:")
        print(f"Sensibilité (Recall positif): {sensitivity:.4f}")
        print(f"Spécificité (Recall négatif): {specificity:.4f}")
        print(f"Précision: {precision:.4f}")
        print(f"F1-Score: {f1_score:.4f}")
        print(f"AUC-ROC: {auc_roc:.4f}")

        self.plot_confusion_matrix(cm)
        self.plot_roc_curve(true_labels, probabilities)

    def plot_confusion_matrix(self, cm):
        """Plot la matrice de confusion"""
        plt.figure(figsize=(8, 6))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                   xticklabels=['Normal', 'Pathologique'],
                   yticklabels=['Normal', 'Pathologique'])
        plt.title('Matrice de Confusion - Détection Médicale')
        plt.ylabel('Vrai label')
        plt.xlabel('Prédiction')
        plt.show()

print(" Évaluateur médical créé!")

 Évaluateur médical créé!


In [47]:
def main():
    """
    Pipeline complet du système de détection médicale
    """
    print(" LANCEMENT DU SYSTÈME DE DÉTECTION MÉDICALE IA")

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f" Device utilisé: {device}")

    print("\n1.  Préparation des données...")

    train_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomRotation(10),
        transforms.Normalize(mean=[0.5], std=[0.5])
    ])

    val_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5], std=[0.5])
    ])

    print("\n2.  Initialisation des modèles...")

    classification_model = MedicalClassifier(num_classes=2).to(device)

    segmentation_model = UNetSegmentation(in_channels=1, out_channels=1).to(device)

    print(f" Modèle classification: {sum(p.numel() for p in classification_model.parameters()):,} paramètres")
    print(f" Modèle segmentation: {sum(p.numel() for p in segmentation_model.parameters()):,} paramètres")

    print("\n3.  Configuration de l'entraînement...")

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(classification_model.parameters(), lr=1e-4, weight_decay=1e-5)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)

    print("\n4.  Entraînement des modèles...")

    print("\n5.  Évaluation des modèles...")

    print("\n SYSTÈME MÉDICAL IA PRÊT À L'EMPLOI!")
    print("\n Mes prochaines étapes:")
    print("   - Charger vos données médicales DICOM/NIFTI")
    print("   - Adapter les paths dans MedicalDataset")
    print("   - Lancer l'entraînement complet")
    print("   - Déployer le modèle avec une interface web")

if __name__ == "__main__":
    main()

 LANCEMENT DU SYSTÈME DE DÉTECTION MÉDICALE IA
 Device utilisé: cpu

1.  Préparation des données...

2.  Initialisation des modèles...


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


 Modèle classification: 24,558,146 paramètres
 Modèle segmentation: 31,036,481 paramètres

3.  Configuration de l'entraînement...

4.  Entraînement des modèles...

5.  Évaluation des modèles...

 SYSTÈME MÉDICAL IA PRÊT À L'EMPLOI!

 Mes prochaines étapes:
   - Charger vos données médicales DICOM/NIFTI
   - Adapter les paths dans MedicalDataset
   - Lancer l'entraînement complet
   - Déployer le modèle avec une interface web


In [48]:
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd

try:
    import pydicom
    print(" pydicom importé avec succès")
except ImportError:
    print(" pydicom non installé. Exécutez: pip install pydicom")
    pydicom = None

try:
    import nibabel as nib
    print(" nibabel importé avec succès")
except ImportError:
    print(" nibabel non installé. Exécutez: pip install nibabel")
    nib = None

class MedicalDataLoader:
    """
    Chargeur professionnel pour données médicales DICOM et NIFTI
    """

    def __init__(self, base_path="./medical_data"):
        self.base_path = base_path
        self.dicom_data = []
        self.nifti_data = []

    def scan_medical_files(self):
        """Scan le répertoire à la recherche de fichiers médicaux"""
        print(" Scan des fichiers médicaux...")

        dicom_patterns = [
            "**/*.dcm",
            "**/*.DCM",
            "**/*.dicom",
            "**/*.DICOM"
        ]

        nifti_patterns = [
            "**/*.nii",
            "**/*.nii.gz",
            "**/*.NII",
            "**/*.NII.GZ"
        ]

        for pattern in dicom_patterns:
            files = glob.glob(os.path.join(self.base_path, pattern), recursive=True)
            self.dicom_data.extend([(f, self._infer_dicom_label(f)) for f in files])

        for pattern in nifti_patterns:
            files = glob.glob(os.path.join(self.base_path, pattern), recursive=True)
            self.nifti_data.extend([(f, self._infer_nifti_label(f)) for f in files])

        print(f" Fichiers DICOM trouvés: {len(self.dicom_data)}")
        print(f" Fichiers NIFTI trouvés: {len(self.nifti_data)}")

    def _infer_dicom_label(self, file_path):
        """Infère le label à partir du chemin du fichier DICOM"""
        path_lower = file_path.lower()
        if 'normal' in path_lower or 'sain' in path_lower:
            return 0
        elif 'cancer' in path_lower or 'tumeur' in path_lower or 'pathological' in path_lower:
            return 1
        elif 'covid' in path_lower or 'pneumonia' in path_lower:
            return 2
        else:
            return 0

    def _infer_nifti_label(self, file_path):
        """Infère le label à partir du chemin du fichier NIFTI"""
        path_lower = file_path.lower()
        if 'mask' in path_lower:
            return 'mask'
        elif 'tumor' in path_lower or 'lesion' in path_lower:
            return 'tumor'
        else:
            return 'image'

    def load_dicom_series(self, folder_path):
        """Charge une série complète DICOM"""
        if not pydicom:
            print(" pydicom non disponible")
            return None

        try:
            dicom_files = []
            for file in os.listdir(folder_path):
                if file.lower().endswith(('.dcm', '.dicom')):
                    file_path = os.path.join(folder_path, file)
                    dicom = pydicom.dcmread(file_path)
                    dicom_files.append(dicom)

            dicom_files.sort(key=lambda x: float(x.ImagePositionPatient[2]))

            volume = np.stack([d.pixel_array for d in dicom_files])

            print(f" Série DICOM chargée: {volume.shape}")
            return volume

        except Exception as e:
            print(f" Erreur chargement série DICOM: {e}")
            return None

    def load_dicom_image(self, file_path):
        """Charge une image DICOM individuelle"""
        if not pydicom:
            print(" pydicom non disponible")
            return None

        try:
            dicom = pydicom.dcmread(file_path)
            metadata = {
                'patient_id': getattr(dicom, 'PatientID', 'Unknown'),
                'study_date': getattr(dicom, 'StudyDate', 'Unknown'),
                'modality': getattr(dicom, 'Modality', 'Unknown'),
                'body_part': getattr(dicom, 'BodyPartExamined', 'Unknown'),
                'pixel_spacing': getattr(dicom, 'PixelSpacing', [1.0, 1.0])
            }

            image = dicom.pixel_array.astype(np.float32)

            if dicom.PhotometricInterpretation == "MONOCHROME1":
                image = np.max(image) - image

            image = (image - np.min(image)) / (np.max(image) - np.min(image) + 1e-8)

            return image, metadata

        except Exception as e:
            print(f" Erreur chargement DICOM {file_path}: {e}")
            return None, None

    def load_nifti_volume(self, file_path):
        """Charge un volume NIFTI (IRM, CT scan)"""
        if not nib:
            print(" nibabel non disponible")
            return None

        try:
            nifti = nib.load(file_path)
            volume = nifti.get_fdata()
            affine = nifti.affine
            header = nifti.header

            metadata = {
                'shape': volume.shape,
                'data_type': header.get_data_dtype(),
                'voxel_sizes': header.get_zooms(),
                'affine': affine
            }

            print(f" NIFTI chargé: {volume.shape}, type: {volume.dtype}")
            return volume, metadata

        except Exception as e:
            print(f" Erreur chargement NIFTI {file_path}: {e}")
            return None, None

    def preprocess_medical_image(self, image, target_size=(256, 256)):
        """Prétraitement standard pour images médicales"""
        import cv2

        if len(image.shape) == 3:
            processed = np.zeros((image.shape[0], target_size[0], target_size[1]))
            for i in range(image.shape[0]):
                slice_img = cv2.resize(image[i], target_size)
                processed[i] = slice_img
        else:
            processed = cv2.resize(image, target_size)

        processed = np.clip(processed, 0, 1)

        return processed

    def visualize_medical_data(self, max_samples=5):
        """Visualise les données médicales chargées"""
        fig, axes = plt.subplots(2, max_samples, figsize=(15, 6))


        print("  Visualisation des données DICOM...")
        for i, (dicom_path, label) in enumerate(self.dicom_data[:max_samples]):
            image, metadata = self.load_dicom_image(dicom_path)
            if image is not None:
                axes[0, i].imshow(image, cmap='gray')
                axes[0, i].set_title(f'DICOM: {label}\n{metadata["modality"]}')
                axes[0, i].axis('off')

        print("  Visualisation des données NIFTI...")
        nifti_samples = [x for x in self.nifti_data if x[1] != 'mask'][:max_samples]
        for i, (nifti_path, label) in enumerate(nifti_samples):
            volume, metadata = self.load_nifti_volume(nifti_path)
            if volume is not None:
                slice_idx = volume.shape[2] // 2 if len(volume.shape) == 3 else 0
                slice_img = volume[:, :, slice_idx] if len(volume.shape) == 3 else volume

                axes[1, i].imshow(slice_img, cmap='gray')
                axes[1, i].set_title(f'NIFTI: {label}\n{volume.shape}')
                axes[1, i].axis('off')

        plt.tight_layout()
        plt.show()

def demo_medical_loader():
    """Démonstration du chargement de données médicales"""

    loader = MedicalDataLoader("./medical_data")

    loader.scan_medical_files()

    if loader.dicom_data or loader.nifti_data:
        loader.visualize_medical_data()

        if loader.dicom_data:
            print("\n" + "="*50)
            print(" EXEMPLE DICOM DÉTAILLÉ")
            print("="*50)

            sample_dicom = loader.dicom_data[0][0]
            image, metadata = loader.load_dicom_image(sample_dicom)

            if image is not None:
                print(f" Dimensions image: {image.shape}")
                print(f" Plage valeurs: [{np.min(image):.3f}, {np.max(image):.3f}]")
                print(f"  Métadonnées: {metadata}")

        if loader.nifti_data:
            print("\n" + "="*50)
            print(" EXEMPLE NIFTI DÉTAILLÉ")
            print("="*50)

            sample_nifti = loader.nifti_data[0][0]
            volume, metadata = loader.load_nifti_volume(sample_nifti)

            if volume is not None:
                print(f" Dimensions volume: {volume.shape}")
                print(f" Type données: {volume.dtype}")
                print(f" Taille voxels: {metadata['voxel_sizes']}")

    else:
        print(" Aucune donnée médicale trouvée.")
        print(" Structure attendue:")
        print("   medical_data/")
        print("   ├── DICOM/")
        print("   │   ├── normal/*.dcm")
        print("   │   └── pathological/*.dcm")
        print("   └── NIFTI/")
        print("       ├── images/*.nii.gz")
        print("       └── masks/*.nii.gz")
def create_sample_medical_data():
    """Crée des données médicales sample pour tester"""
    sample_dir = "./medical_data_sample"
    os.makedirs(sample_dir, exist_ok=True)

    print(" Création de données sample...")


    print(f" Répertoire sample créé: {sample_dir}")

if __name__ == "__main__":
    create_sample_medical_data()

    demo_medical_loader()

 pydicom importé avec succès
 nibabel importé avec succès
 Création de données sample...
 Répertoire sample créé: ./medical_data_sample
 Scan des fichiers médicaux...
 Fichiers DICOM trouvés: 0
 Fichiers NIFTI trouvés: 0
 Aucune donnée médicale trouvée.
 Structure attendue:
   medical_data/
   ├── DICOM/
   │   ├── normal/*.dcm
   │   └── pathological/*.dcm
   └── NIFTI/
       ├── images/*.nii.gz
       └── masks/*.nii.gz


In [49]:
def get_public_medical_datasets():
    """Liste des datasets médicaux publics"""
    datasets = {
        'DICOM': [
            'COVID-19 Chest X-Ray (Kaggle)',
            'RSNA Pneumonia Detection',
            'ChestX-ray8 (NIH)'
        ],
        'NIFTI': [
            'BraTS (Brain Tumor Segmentation)',
            'LiTS (Liver Tumor Segmentation)',
            'MSD (Medical Segmentation Decathlon)'
        ]
    }

    print(" Datasets médicaux publics recommandés:")
    for modality, ds_list in datasets.items():
        print(f"\n {modality}:")
        for ds in ds_list:
            print(f"   • {ds}")

    return datasets

get_public_medical_datasets()

 Datasets médicaux publics recommandés:

 DICOM:
   • COVID-19 Chest X-Ray (Kaggle)
   • RSNA Pneumonia Detection
   • ChestX-ray8 (NIH)

 NIFTI:
   • BraTS (Brain Tumor Segmentation)
   • LiTS (Liver Tumor Segmentation)
   • MSD (Medical Segmentation Decathlon)


{'DICOM': ['COVID-19 Chest X-Ray (Kaggle)',
  'RSNA Pneumonia Detection',
  'ChestX-ray8 (NIH)'],
 'NIFTI': ['BraTS (Brain Tumor Segmentation)',
  'LiTS (Liver Tumor Segmentation)',
  'MSD (Medical Segmentation Decathlon)']}

In [50]:
!kaggle datasets download -d tawsifurrahman/covid19-radiography-database

Dataset URL: https://www.kaggle.com/datasets/tawsifurrahman/covid19-radiography-database
License(s): copyright-authors
 99% 773M/778M [00:09<00:00, 85.2MB/s]
100% 778M/778M [00:09<00:00, 89.5MB/s]


In [51]:
demo_medical_loader()

 Scan des fichiers médicaux...
 Fichiers DICOM trouvés: 0
 Fichiers NIFTI trouvés: 0
 Aucune donnée médicale trouvée.
 Structure attendue:
   medical_data/
   ├── DICOM/
   │   ├── normal/*.dcm
   │   └── pathological/*.dcm
   └── NIFTI/
       ├── images/*.nii.gz
       └── masks/*.nii.gz


In [52]:
pip install kaggle --upgrade

In [53]:
import os
kaggle_path = os.path.expanduser('~/.kaggle/kaggle.json')
if not os.path.exists(kaggle_path):
    print(" kaggle.json non trouvé. Suivez les instructions de configuration.")

 kaggle.json non trouvé. Suivez les instructions de configuration.


In [55]:
# Create the .kaggle directory if it doesn't exist
!mkdir -p ~/.kaggle


In [56]:
# Move the uploaded kaggle.json to the .kaggle directory
!mv kaggle.json ~/.kaggle/


mv: cannot stat 'kaggle.json': No such file or directory


In [57]:
# Set appropriate permissions for the kaggle.json file
!chmod 600 ~/.kaggle/kaggle.json

print("Kaggle API key configured successfully!")


chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Kaggle API key configured successfully!


In [58]:
# Verify the configuration by trying to list Kaggle datasets
!kaggle datasets list -s covid19


Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method. See setup instructions at https://github.com/Kaggle/kaggle-api/


In [59]:
# Move the uploaded kaggle.json to the .kaggle directory
!mv kaggle.json ~/.kaggle/

mv: cannot stat 'kaggle.json': No such file or directory


In [60]:
# Set appropriate permissions for the kaggle.json file
!chmod 600 ~/.kaggle/kaggle.json

print("Kaggle API key configured successfully!")

chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Kaggle API key configured successfully!


In [61]:
# Verify the configuration by trying to list Kaggle datasets
!kaggle datasets list -s covid19

Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method. See setup instructions at https://github.com/Kaggle/kaggle-api/


In [62]:
# Move the uploaded kaggle.json to the .kaggle directory
!mv kaggle.json ~/.kaggle/

mv: cannot stat 'kaggle.json': No such file or directory


In [63]:
# Set appropriate permissions for the kaggle.json file
!chmod 600 ~/.kaggle/kaggle.json

print("Kaggle API key configured successfully!")

chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Kaggle API key configured successfully!


In [64]:
# Verify the configuration by trying to list Kaggle datasets
!kaggle datasets list -s covid19

Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method. See setup instructions at https://github.com/Kaggle/kaggle-api/
